<a href="https://colab.research.google.com/github/MaYuetong/MaYuetong/blob/main/%E6%9C%80%E4%BD%B3%E5%8C%85%E8%A3%85%E6%8E%A8%E8%8D%90%E5%B7%A5%E5%85%B7%E7%AE%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
from itertools import permutations
import numpy as np
from google.colab import files

# 让用户上传文件到暂存端
uploaded = files.upload()  # 上传文件
file_path = list(uploaded.keys())[0]  # 获取上传文件的路径

# 从文件中读取数据
xls = pd.ExcelFile(file_path)

# 根据文件名或手动指定名称
# 假设文件名包含名称
warehouse_name = file_path.split('-')[0]  # 从文件名提取名称

# 读取数据
sheet1 = pd.read_excel(xls, '基础数据')  # 每个仓库的基础单据
sheet2 = pd.read_excel(xls, '仓库原有箱型方案')  # 当前箱型方案
sheet3 = pd.read_excel(xls, '新箱型方案')  # 新版箱型方案

# 重命名列名，去掉单位
sheet2.rename(columns={'内径长度(cm)': '内径长度', '内径宽度(cm)': '内径宽度', '内径高度(cm)': '内径高度', '容器毛重(KG)': '原仓容器重量'}, inplace=True)
sheet3.rename(columns={'内径长度(cm)': '内径长度', '内径宽度(cm)': '内径宽度', '内径高度(cm)': '内径高度', '容器毛重(KG)': '新版容器重量'}, inplace=True)

# 标记原箱发货
sheet1['发货方式'] = 1  # 默认计算摆放
for order_id, group in sheet1.groupby('出库单号'):
    material_counts = group['物料号'].value_counts()
    for material, count in material_counts.items():
        box_rule = group[group['物料号'] == material]['箱规'].iloc[0]
        if count >= box_rule:
            sheet1.loc[sheet1['出库单号'] == order_id, '发货方式'] = 0  # 直接发原箱

# 计算单包裹重量
def calculate_package_weight(group, selected_box_code, box_df, weight_col):
    # 获取箱子重量
    box_weight = box_df.loc[box_df['容器编码'] == selected_box_code, weight_col].values
    box_weight = box_weight[0] if len(box_weight) > 0 else 0  # 防止取不到值

    # 计算单包裹重量 = 彩盒重量总和 + 容器重量
    package_weight = group['彩盒重量（公斤）'].sum() + box_weight
    return package_weight

# 贪婪前向树算法进行摆放优化，先尝试体积优先，再尝试底面积优先
def greedy_forward_tree(items, box):
    packed_items = []
    remaining_space = [(0, 0, 0, box.内径长度, box.内径宽度, box.内径高度)]

    for item in items:
        for i, (x, y, z, l, w, h) in enumerate(remaining_space):
            if item[0] <= l and item[1] <= w and item[2] <= h:
                packed_items.append((x, y, z, item[0], item[1], item[2]))
                remaining_space.pop(i)
                remaining_space.append((x + item[0], y, z, l - item[0], w, h))
                remaining_space.append((x, y + item[1], z, l, w - item[1], h))
                remaining_space.append((x, y, z + item[2], l, w, h - item[2]))
                break

    return packed_items, sum(i[3] * i[4] * i[5] for i in packed_items) / (box.内径长度 * box.内径宽度 * box.内径高度)

# 计算最优箱型
def find_best_box(items, boxes):
    best_fit = None
    best_utilization = 0

    sorted_items_vol = sorted(items, key=lambda x: (x[0] * x[1] * x[2]), reverse=True)
    for box in boxes.itertuples():
        packed_items, utilization = greedy_forward_tree(sorted_items_vol, box)
        if utilization > best_utilization and utilization <1:
            best_utilization = utilization
            best_fit = box.容器编码

    if best_utilization < 0.85:
        sorted_items_area = sorted(items, key=lambda x: (x[1] * x[0], max(x[0], x[1], x[2])), reverse=True)
        for box in boxes.itertuples():
            packed_items, utilization = greedy_forward_tree(sorted_items_area, box)
            if utilization > best_utilization and utilization <= 0.9:
                best_utilization = utilization
                best_fit = box.容器编码

    return best_fit

# 计算每个出库单的最优箱型
optimal_boxes = []
for order_id, group in sheet1.groupby('出库单号'):
    if group['发货方式'].iloc[0] == 0:
        optimal_boxes.append([order_id, '原箱发货', '原箱发货', 0, 0])
        continue

    items = list(zip(group['彩盒长'], group['彩盒宽'], group['彩盒高']))
    best_box_current = find_best_box(items, sheet2)  # 只在当前箱型方案中选择
    best_box_new = find_best_box(items, sheet3)  # 只在新版箱型方案中选择

    package_weight_current = calculate_package_weight(group, best_box_current, sheet2, '原仓容器重量')
    package_weight_new = calculate_package_weight(group, best_box_new, sheet3, '新版容器重量')

    optimal_boxes.append([order_id, best_box_current if best_box_current else '无合适箱型',
                          best_box_new if best_box_new else '无合适箱型', package_weight_current, package_weight_new])

# 输出最优箱型推荐
output_path = f'/content/{warehouse_name}_optimal_boxes.xlsx'
df_optimal_boxes = pd.DataFrame(optimal_boxes, columns=['出库单号', '原仓推荐箱型', '新版推荐箱型', '原仓包裹重量（kg）', '新版包裹重量（kg）'])
df_optimal_boxes.to_excel(output_path, index=False)

# 让用户下载计算结果
files.download(output_path)

print(f"最优箱型推荐计算完成，结果已保存至 {output_path}，请下载文件。")

Saving 演示案例_最佳包装推荐.xlsx to 演示案例_最佳包装推荐 (1).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

最优箱型推荐计算完成，结果已保存至 /content/演示案例_最佳包装推荐 (1).xlsx_optimal_boxes.xlsx，请下载文件。
